In [1]:
#Imports
import pandas as pd
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib
import numpy as np

In [2]:
# Watermark
print('Nathan Young\nJunior Data Analyst\nCenter for the Study of Free Enterprise')
%load_ext watermark
%watermark -a "Western Carolina University" -u -d -p pandas

Nathan Young
Junior Data Analyst
Center for the Study of Free Enterprise
Western Carolina University 
last updated: 2020-01-28 

pandas 0.25.3


In [3]:
# Create Backups
#df_backup = pd.read_csv('./Updates/STG_ZLLW_County_MedianListingPrice_AllHomes.txt')
#df_backup.to_csv('./Backups/STG_ZLLW_County_MedianListingPrice_AllHomes_BACKUP.txt')

In [4]:
#Load Land data
df_mlp = pd.read_csv('http://files.zillowstatic.com/research/public/County/County_MedianListingPrice_AllHomes.csv',
                     encoding='ISO-8859-1')

#Display table to ensure data loaded correctly
df_mlp.head()

,RegionName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,SizeRank,2010-01,2010-02,2010-03,2010-04,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,Los Angeles County,CA,Los Angeles-Long Beach-Anaheim,6,37,1,399000.0,389000.0,399000.0,398884.0,...,665000.0,675000.0,679000.0,689000.0,695000.0,689000.0,689000.0,690950.0,695000.0,695000.0
1,Cook County,IL,Chicago-Naperville-Elgin,17,31,2,244900.0,249000.0,249000.0,249000.0,...,299000.0,300000.0,300000.0,299900.0,299000.0,292900.0,289999.0,288000.0,284900.0,279900.0
2,Harris County,TX,Houston-The Woodlands-Sugar Land,48,201,3,149900.0,152900.0,154788.5,154900.0,...,280000.0,286000.0,284995.0,283990.0,279000.0,275000.0,274999.0,271990.0,274000.0,273000.0
3,Maricopa County,AZ,Phoenix-Mesa-Scottsdale,4,13,4,179900.0,176100.0,174900.0,169900.0,...,315000.0,318990.0,319000.0,319900.0,317230.0,315000.0,319000.0,319900.0,320000.0,324900.0
4,San Diego County,CA,San Diego-Carlsbad,6,73,5,399000.0,399000.0,399500.0,399990.0,...,643400.0,650000.0,659800.0,655495.0,649990.0,650000.0,657900.0,649900.0,650000.0,650000.0


In [5]:
#Filter data to NC
filter1 = df_mlp['State'] == "NC"
df_mlp_nc = df_mlp[filter1]

#Check to ensure filter worked
df_mlp_nc.head(5)

,RegionName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,SizeRank,2010-01,2010-02,2010-03,2010-04,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
48,Mecklenburg County,NC,Charlotte-Concord-Gastonia,37,119,49,189900.0,194500.0,190000.0,194000.0,...,318000.0,324990.0,321890.5,320000.0,318759.5,314990.0,315000.0,315000.0,318999.5,318999.0
55,Wake County,NC,Raleigh,37,183,56,247945.0,245000.0,239900.0,239900.0,...,359000.0,364900.0,359900.0,359900.0,358245.0,357127.0,358935.0,352900.0,350000.0,356423.0
134,Guilford County,NC,Greensboro-High Point,37,81,135,159900.0,159900.0,159900.0,159900.0,...,208000.0,212900.0,214250.0,214189.5,214900.0,207900.0,209900.0,214900.0,209990.0,209900.0
183,Forsyth County,NC,Winston-Salem,37,67,184,154900.0,156800.0,159900.0,159900.0,...,215000.0,224900.0,219900.0,225000.0,227400.0,220000.0,217200.0,220000.0,225000.0,225000.0
198,Cumberland County,NC,Fayetteville,37,51,199,NaN,NaN,NaN,NaN,...,165000.0,172000.0,169500.0,169000.0,168000.0,169000.0,163000.0,161430.0,162950.0,162700.0


In [6]:
#View data types of dataframe
df_mlp_nc.dtypes

RegionName            object
State                 object
Metro                 object
StateCodeFIPS          int64
MunicipalCodeFIPS      int64
                      ...   
2019-08              float64
2019-09              float64
2019-10              float64
2019-11              float64
2019-12              float64
Length: 126, dtype: object

In [7]:
#Change MunicipalCodeFIPS dtype to add leading 0's
df_mlp_nc.loc[ :, 'MunicipalCodeFIPS'] = df_mlp_nc['MunicipalCodeFIPS'].astype(str)
df_mlp_nc.dtypes

c:\users\natha\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


RegionName            object
State                 object
Metro                 object
StateCodeFIPS          int64
MunicipalCodeFIPS     object
                      ...   
2019-08              float64
2019-09              float64
2019-10              float64
2019-11              float64
2019-12              float64
Length: 126, dtype: object

In [8]:
#Add leading 0's and check to ensure they were added
df_mlp_nc.loc[ :, 'MunicipalCodeFIPS'] = df_mlp_nc['MunicipalCodeFIPS'].str.zfill(3)
df_mlp_nc.head(5)

,RegionName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,SizeRank,2010-01,2010-02,2010-03,2010-04,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
48,Mecklenburg County,NC,Charlotte-Concord-Gastonia,37,119,49,189900.0,194500.0,190000.0,194000.0,...,318000.0,324990.0,321890.5,320000.0,318759.5,314990.0,315000.0,315000.0,318999.5,318999.0
55,Wake County,NC,Raleigh,37,183,56,247945.0,245000.0,239900.0,239900.0,...,359000.0,364900.0,359900.0,359900.0,358245.0,357127.0,358935.0,352900.0,350000.0,356423.0
134,Guilford County,NC,Greensboro-High Point,37,081,135,159900.0,159900.0,159900.0,159900.0,...,208000.0,212900.0,214250.0,214189.5,214900.0,207900.0,209900.0,214900.0,209990.0,209900.0
183,Forsyth County,NC,Winston-Salem,37,067,184,154900.0,156800.0,159900.0,159900.0,...,215000.0,224900.0,219900.0,225000.0,227400.0,220000.0,217200.0,220000.0,225000.0,225000.0
198,Cumberland County,NC,Fayetteville,37,051,199,NaN,NaN,NaN,NaN,...,165000.0,172000.0,169500.0,169000.0,168000.0,169000.0,163000.0,161430.0,162950.0,162700.0


In [9]:
# Set Index to Region Name
df_mlp_nc.set_index(df_mlp_nc['RegionName'], inplace = True)
df_mlp_nc

,RegionName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,SizeRank,2010-01,2010-02,2010-03,2010-04,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
RegionName,,,,,,,,,,,,,,,,,,,,,
Mecklenburg County,Mecklenburg County,NC,Charlotte-Concord-Gastonia,37,119,49,189900.0,194500.0,190000.0,194000.0,...,318000.0,324990.0,321890.5,320000.0,318759.5,314990.0,315000.0,315000.0,318999.5,318999.0
Wake County,Wake County,NC,Raleigh,37,183,56,247945.0,245000.0,239900.0,239900.0,...,359000.0,364900.0,359900.0,359900.0,358245.0,357127.0,358935.0,352900.0,350000.0,356423.0
Guilford County,Guilford County,NC,Greensboro-High Point,37,081,135,159900.0,159900.0,159900.0,159900.0,...,208000.0,212900.0,214250.0,214189.5,214900.0,207900.0,209900.0,214900.0,209990.0,209900.0
Forsyth County,Forsyth County,NC,Winston-Salem,37,067,184,154900.0,156800.0,159900.0,159900.0,...,215000.0,224900.0,219900.0,225000.0,227400.0,220000.0,217200.0,220000.0,225000.0,225000.0
Cumberland County,Cumberland County,NC,Fayetteville,37,051,199,NaN,NaN,NaN,NaN,...,165000.0,172000.0,169500.0,169000.0,168000.0,169000.0,163000.0,161430.0,162950.0,162700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Alleghany County,Alleghany County,NC,NaN,37,005,2093,229000.0,227450.0,225900.0,235250.0,...,229000.0,237000.0,238000.0,244250.0,249000.0,248250.0,237000.0,241750.0,238500.0,228450.0
Clay County,Clay County,NC,NaN,37,043,2115,239800.0,239000.0,239000.0,239900.0,...,249949.5,251000.0,251000.0,259000.0,259900.0,259000.0,265000.0,259900.0,265000.0,269000.0
Camden County,Camden County,NC,Elizabeth City,37,029,2137,275000.0,274900.0,259900.0,259900.0,...,289900.0,285900.0,285900.0,279900.0,285900.0,289000.0,289000.0,289900.0,294900.0,295900.0


In [10]:
# Drop Region Name column
df_mlp_nc.drop('RegionName', axis = 1, inplace = True)
df_mlp_nc

c:\users\natha\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,SizeRank,2010-01,2010-02,2010-03,2010-04,2010-05,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
RegionName,,,,,,,,,,,,,,,,,,,,,
Mecklenburg County,NC,Charlotte-Concord-Gastonia,37,119,49,189900.0,194500.0,190000.0,194000.0,190000.0,...,318000.0,324990.0,321890.5,320000.0,318759.5,314990.0,315000.0,315000.0,318999.5,318999.0
Wake County,NC,Raleigh,37,183,56,247945.0,245000.0,239900.0,239900.0,244900.0,...,359000.0,364900.0,359900.0,359900.0,358245.0,357127.0,358935.0,352900.0,350000.0,356423.0
Guilford County,NC,Greensboro-High Point,37,081,135,159900.0,159900.0,159900.0,159900.0,159900.0,...,208000.0,212900.0,214250.0,214189.5,214900.0,207900.0,209900.0,214900.0,209990.0,209900.0
Forsyth County,NC,Winston-Salem,37,067,184,154900.0,156800.0,159900.0,159900.0,158354.0,...,215000.0,224900.0,219900.0,225000.0,227400.0,220000.0,217200.0,220000.0,225000.0,225000.0
Cumberland County,NC,Fayetteville,37,051,199,NaN,NaN,NaN,NaN,NaN,...,165000.0,172000.0,169500.0,169000.0,168000.0,169000.0,163000.0,161430.0,162950.0,162700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Alleghany County,NC,NaN,37,005,2093,229000.0,227450.0,225900.0,235250.0,229900.0,...,229000.0,237000.0,238000.0,244250.0,249000.0,248250.0,237000.0,241750.0,238500.0,228450.0
Clay County,NC,NaN,37,043,2115,239800.0,239000.0,239000.0,239900.0,239900.0,...,249949.5,251000.0,251000.0,259000.0,259900.0,259000.0,265000.0,259900.0,265000.0,269000.0
Camden County,NC,Elizabeth City,37,029,2137,275000.0,274900.0,259900.0,259900.0,250000.0,...,289900.0,285900.0,285900.0,279900.0,285900.0,289000.0,289000.0,289900.0,294900.0,295900.0


In [11]:
#Save to csv file for export in Excel
#df_mlp_nc.to_csv('./Updates/STG_ZLLW_County_MedianListingPrice_AllHomes.txt', sep ='\t')

In [12]:
#Reset Index for upload to database
df_mlp_nc = df_mlp_nc.reset_index()    

In [13]:
#Fill NaN values for upload to database
df_mlp_nc['Metro'] = df_mlp_nc['Metro'].replace(np.nan,'', regex=True)

column_list = df_mlp_nc.columns.values
for i in column_list:
    df_mlp_nc.loc[df_mlp_nc[i].isnull(),i]=0

In [14]:
#Connect to database and create cursor
con = pyodbc.connect('Driver={SQL Server};'
                      'Server=TITANIUM-BOOK;'
                      'Database=DataDashboard;'
                      'Trusted_Connection=yes;')

c = con.cursor()

In [15]:
#Verify data is in database
c.execute('select top(1) * from dbo.STG_ZLLW_County_MedianListingPrice_AllHomes')
for row in c:
    print(row)

('Mecklenburg County', 'NC', 'Charlotte-Concord-Gastonia', 37, '119', 49, 189900.0, 194500.0, 190000.0, 194000.0, 190000.0, 190000.0, 189750.0, 189900.0, 185900.0, 185000.0, 184900.0, 178000.0, 175000.0, 175000.0, 175000.0, 179407.5, 179900.0, 179900.0, 179000.0, 178999.5, 175000.0, 175000.0, 174900.0, 173995.0, 174900.0, 177930.0, 182500.0, 184900.0, 184900.0, 184900.0, 179900.0, 185000.0, 182000.0, 184900.0, 183000.0, 185000.0, 188900.0, 199500.0, 206900.0, 211995.0, 214500.0, 214375.0, 210000.0, 209990.0, 209900.0, 209000.0, 205000.0, 200000.0, 199000.0, 200000.0, 215000.0, 220999.5, 224500.0, 224875.5, 222227.0, 219900.0, 219900.0, 219945.0, 219995.0, 219990.0, 225000.0, 230000.0, 233000.0, 242990.0, 249970.0, 250000.0, 251490.0, 250000.0, 249000.0, 254345.0, 254900.0, 250000.0, 264450.0, 274900.0, 279900.0, 285990.0, 289900.0, 289000.0, 289900.0, 281990.0, 284900.0, 286390.0, 285000.0, 289900.0, 288200.0, 285000.0, 294500.0, 300000.0, 304990.0, 305000.0, 299000.0, 295500.0, 299900

In [16]:
#Drop old backup table
c.execute('drop table STG_ZLLW_County_MedianListingPrice_AllHomes_BACKUP')

In [17]:
#Create new backup
c.execute("sp_rename 'dbo.STG_ZLLW_County_MedianListingPrice_AllHomes','STG_ZLLW_County_MedianListingPrice_AllHomes_BACKUP';")
con.commit()

In [18]:
#Verify backups are created
c.execute('select top(1) * from dbo.STG_ZLLW_County_MedianListingPrice_AllHomes_BACKUP')
for row in c:
    print(row)

('Mecklenburg County', 'NC', 'Charlotte-Concord-Gastonia', 37, '119', 49, 189900.0, 194500.0, 190000.0, 194000.0, 190000.0, 190000.0, 189750.0, 189900.0, 185900.0, 185000.0, 184900.0, 178000.0, 175000.0, 175000.0, 175000.0, 179407.5, 179900.0, 179900.0, 179000.0, 178999.5, 175000.0, 175000.0, 174900.0, 173995.0, 174900.0, 177930.0, 182500.0, 184900.0, 184900.0, 184900.0, 179900.0, 185000.0, 182000.0, 184900.0, 183000.0, 185000.0, 188900.0, 199500.0, 206900.0, 211995.0, 214500.0, 214375.0, 210000.0, 209990.0, 209900.0, 209000.0, 205000.0, 200000.0, 199000.0, 200000.0, 215000.0, 220999.5, 224500.0, 224875.5, 222227.0, 219900.0, 219900.0, 219945.0, 219995.0, 219990.0, 225000.0, 230000.0, 233000.0, 242990.0, 249970.0, 250000.0, 251490.0, 250000.0, 249000.0, 254345.0, 254900.0, 250000.0, 264450.0, 274900.0, 279900.0, 285990.0, 289900.0, 289000.0, 289900.0, 281990.0, 284900.0, 286390.0, 285000.0, 289900.0, 288200.0, 285000.0, 294500.0, 300000.0, 304990.0, 305000.0, 299000.0, 295500.0, 299900

In [19]:
c.execute('''USE [DataDashboard]

SET ANSI_NULLS ON

SET QUOTED_IDENTIFIER ON

CREATE TABLE [dbo].[STG_ZLLW_County_MedianListingPrice_AllHomes](
	[RegionName] [varchar](40) NULL,
	[State] [varchar](2) NULL,
	[Metro] [varchar](40) NULL,
	[StateCodeFIPS] [varchar](2) NULL,
	[MunicipalCodeFIPS] [varchar](3) NULL,
	[SizeRank] [smallint] NULL,
	[2010-01] [float] NULL,
	[2010-02] [float] NULL,
	[2010-03] [float] NULL,
	[2010-04] [float] NULL,
	[2010-05] [float] NULL,
	[2010-06] [float] NULL,
	[2010-07] [float] NULL,
	[2010-08] [float] NULL,
	[2010-09] [float] NULL,
	[2010-10] [float] NULL,
	[2010-11] [float] NULL,
	[2010-12] [float] NULL,
	[2011-01] [float] NULL,
	[2011-02] [float] NULL,
	[2011-03] [float] NULL,
	[2011-04] [float] NULL,
	[2011-05] [float] NULL,
	[2011-06] [float] NULL,
	[2011-07] [float] NULL,
	[2011-08] [float] NULL,
	[2011-09] [float] NULL,
	[2011-10] [float] NULL,
	[2011-11] [float] NULL,
	[2011-12] [float] NULL,
	[2012-01] [float] NULL,
	[2012-02] [float] NULL,
	[2012-03] [float] NULL,
	[2012-04] [float] NULL,
	[2012-05] [float] NULL,
	[2012-06] [float] NULL,
	[2012-07] [float] NULL,
	[2012-08] [float] NULL,
	[2012-09] [float] NULL,
	[2012-10] [float] NULL,
	[2012-11] [float] NULL,
	[2012-12] [float] NULL,
	[2013-01] [float] NULL,
	[2013-02] [float] NULL,
	[2013-03] [float] NULL,
	[2013-04] [float] NULL,
	[2013-05] [float] NULL,
	[2013-06] [float] NULL,
	[2013-07] [float] NULL,
	[2013-08] [float] NULL,
	[2013-09] [float] NULL,
	[2013-10] [float] NULL,
	[2013-11] [float] NULL,
	[2013-12] [float] NULL,
	[2014-01] [float] NULL,
	[2014-02] [float] NULL,
	[2014-03] [float] NULL,
	[2014-04] [float] NULL,
	[2014-05] [float] NULL,
	[2014-06] [float] NULL,
	[2014-07] [float] NULL,
	[2014-08] [float] NULL,
	[2014-09] [float] NULL,
	[2014-10] [float] NULL,
	[2014-11] [float] NULL,
	[2014-12] [float] NULL,
	[2015-01] [float] NULL,
	[2015-02] [float] NULL,
	[2015-03] [float] NULL,
	[2015-04] [float] NULL,
	[2015-05] [float] NULL,
	[2015-06] [float] NULL,
	[2015-07] [float] NULL,
	[2015-08] [float] NULL,
	[2015-09] [float] NULL,
	[2015-10] [float] NULL,
	[2015-11] [float] NULL,
	[2015-12] [float] NULL,
	[2016-01] [float] NULL,
	[2016-02] [float] NULL,
	[2016-03] [float] NULL,
	[2016-04] [float] NULL,
	[2016-05] [float] NULL,
	[2016-06] [float] NULL,
	[2016-07] [float] NULL,
	[2016-08] [float] NULL,
	[2016-09] [float] NULL,
	[2016-10] [float] NULL,
	[2016-11] [float] NULL,
	[2016-12] [float] NULL,
	[2017-01] [float] NULL,
	[2017-02] [float] NULL,
	[2017-03] [float] NULL,
	[2017-04] [float] NULL,
	[2017-05] [float] NULL,
	[2017-06] [float] NULL,
	[2017-07] [float] NULL,
	[2017-08] [float] NULL,
	[2017-09] [float] NULL,
	[2017-10] [float] NULL,
	[2017-11] [float] NULL,
	[2017-12] [float] NULL,
	[2018-01] [float] NULL,
	[2018-02] [float] NULL,
	[2018-03] [float] NULL,
	[2018-04] [float] NULL,
	[2018-05] [float] NULL,
	[2018-06] [float] NULL,
	[2018-07] [float] NULL,
	[2018-08] [float] NULL,
	[2018-09] [float] NULL,
	[2018-10] [float] NULL,
	[2018-11] [float] NULL,
	[2018-12] [float] NULL,
	[2019-01] [float] NULL,
	[2019-02] [float] NULL,
	[2019-03] [float] NULL,
	[2019-04] [float] NULL,
	[2019-05] [float] NULL,
	[2019-06] [float] NULL,
	[2019-07] [float] NULL,
	[2019-08] [float] NULL,
	[2019-09] [float] NULL,
	[2019-10] [float] NULL,
	[2019-11] [float] NULL,
	[2019-12] [float] NULL,
    [2020-01] [float] NULL,
    [2020-02] [float] NULL,
    [2020-03] [float] NULL,
    [2020-04] [float] NULL,
    [2020-05] [float] NULL,
    [2020-06] [float] NULL,
    [2020-07] [float] NULL,
    [2020-08] [float] NULL,
    [2020-09] [float] NULL,
    [2020-10] [float] NULL,
    [2020-11] [float] NULL,
    [2020-12] [float] NULL
) ON [PRIMARY]''')

In [20]:
con.commit()

In [21]:
from sqlalchemy import create_engine
params = urllib.parse.quote_plus(r'Driver={SQL Server};' 
                                 r'Server=TITANIUM-BOOK;'
                                 r'Database=DataDashboard;'
                                 r'Trusted_Connection=yes;')

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

#df: pandas.dataframe; mTableName:table name in MS SQL
#warning: discard old table if exists
df_mlp_nc.to_sql('STG_ZLLW_County_MedianListingPrice_AllHomes', con=engine, if_exists='replace', index=False)